In [ ]:
include("/Users/meesvandartel/Desktop/Coursework/CGT/DeepEWA/FastEWA.jl")
using Flux, .fEWA, Random, IterTools, ProgressMeter, DataFrames, CSV, Statistics

In [ ]:
# this data will be for this specific coordination game with 2 NE and 1 mixed NE.
coord = [[[5 1; 1 4], [5 1; 1 4]], fEWA.find_NE_mixed([[5 1; 1 4], [5 1; 1 4]])]
dom = [[[5 0; 20 1], [5 0; 20 1]], fEWA.find_NE_mixed([[5 0; 20 1], [5 0; 20 1]])]
cyclic = [[[5 1;1 4],[-5 1; 1 -4]], fEWA.find_NE_mixed([[5 1;1 4],[-5 1; 1 -4]])]

game = coord
points = 10000 

α_grid = rand(0.0:0.0001:1.0, points) 
κ_grid = rand(0.0:0.0001:1.0, points)
δ_grid = rand(0.0:0.0001:1.0, points)
β_grid = exp.(rand(0.0:0.001:2.5, points)) # take exp to get the lim to infty effect since β is unbounded above
 combs = [[α_grid[i], κ_grid[i], δ_grid[i], β_grid[i]] for i in 1:points]

#testing
#combs = [[0.0, 1.0, δ_grid[i], β_grid[i]] for i in 1:points]

subset=Int64(floor(0.6*length(combs))) #60/40 train test split, since we have many obs

x_train, x_test = combs[1:subset], combs[(subset+1):end]
y_train, y_test = Vector{Vector{Bool}}(undef, length(x_train)), Vector{Vector{Bool}}(undef, length(x_test))

In [ ]:
# should take about 6min for 1mil obs
@showprogress for i in 1:length(x_train)
    comb=x_train[i]
    α, κ, δ, β = comb
    params = fEWA.init_EWA(;α=α, κ=κ, δ=δ, β=β,game=game)
    sₜ, σ, Qₜ, cycles, pure_NE, mixed_NE, FP  = fEWA.multicat_FastEWA(params)
    y_train[i] = [cycles, pure_NE, mixed_NE, FP]
end

@showprogress for i in 1:length(x_test)
    comb = x_test[i]
    α, κ, δ, β = comb
    params = fEWA.init_EWA(;α=α, κ=κ, δ=δ, β=β,game=game)
    sₜ, σ, Qₜ, cycles, pure_NE, mixed_NE, FP = fEWA.multicat_FastEWA(params)
    y_test[i] = [cycles, pure_NE, mixed_NE, FP]
end

In [ ]:
x_train = hcat([Float32.(x[1:4]) for x in x_train]...)
y_train = hcat([Float32.(x[1:4]) for x in y_train]...)

DeepEWA= Chain(
    Dense(4 => 32, relu),
    Dense(32 => 32, relu),
    Dense(32 => 4))

out1 = DeepEWA(x_train)
probs1 = softmax(out1)


target = Flux.onehotbatch(y_train, 1:4)
loader = Flux.DataLoader((x_train, target), batchsize=64, shuffle=true);

opt_state = Flux.setup(Flux.AdaGrad(), DeepEWA)

losses=[]
@showprogress for epoch in 1:2000
    for xy in loader
        x,y = xy
        loss, grads = Flux.withgradient(DeepEWA) do m
            y_hat = m(x)
            Flux.logitcrossentropy(y_hat, y)
        end
        Flux.update!(opt_state, DeepEWA, grads[1])
        push!(losses, loss)
    end
end

opt_state

In [ ]:
x_test = hcat([Float32.(x[1:4]) for x in x_test]...)
y_test = hcat([Float32.(x[1:4]) for x in y_test]...)


out2 = DeepEWA(x_test)
probs2 = softmax(out2)
mean((probs2[1,:].>0.5).==y_test)

In [ ]:
using Plots  # to draw the above figure

p_true = scatter(x_test[3,:], x_test[4,:], zcolor=y_train, title="True classification", legend=false)
p_done = scatter(x_test[3,:], x_test[4,:], zcolor=probs2[1,:], title="Trained network", legend=false)

plot(p_true, p_done, layout=(1,2), size=(1000,330))


In [ ]:

#=
plot(losses; xaxis=(:log10, "iteration"),
    yaxis="loss", label="per batch")
n = length(loader)
plot!(n:n:length(losses), mean.(Iterators.partition(losses, n)),
    label="epoch mean", dpi=200)
    =#
